In [6]:
!pip install -r /content/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-5lnl6ahv
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-5lnl6ahv
  Resolved https://github.com/huggingface/transformers.git to commit 17a55534f5e5df10ac4804d4270bf6b8cc24998d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-qto8g12f
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-qto8g12f
  Resolved https://github.com/huggingface/peft.git to commit 3714aa2fff158fdfa637b2b65952580801d890b2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... 

In [7]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import torch
import transformers
import peft


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('http'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

In [28]:
!unzip /content/new_trained.zip

Archive:  /content/new_trained.zip
  inflating: new_trained/adapter_config.json  
  inflating: new_trained/adapter_model.bin  
   creating: new_trained/runs/
   creating: new_trained/runs/May28_10-50-08_f2c025dc96ff/
   creating: new_trained/runs/May28_10-50-08_f2c025dc96ff/1685271008.6259046/
  inflating: new_trained/runs/May28_10-50-08_f2c025dc96ff/1685271008.6259046/events.out.tfevents.1685271008.f2c025dc96ff.13629.1  
  inflating: new_trained/runs/May28_10-50-08_f2c025dc96ff/events.out.tfevents.1685271008.f2c025dc96ff.13629.0  


In [34]:
model = transformers.AutoModelForCausalLM.from_pretrained(
    'cerebras/Cerebras-GPT-1.3B', 
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map='auto'
)

In [35]:
model.config

GPT2Config {
  "_name_or_path": "cerebras/Cerebras-GPT-1.3B",
  "activation_function": "gelu",
  "attn_pdrop": 0.0,
  "bos_token_id": 50256,
  "embd_pdrop": 0.0,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_embd": 2048,
  "n_head": 16,
  "n_inner": 8192,
  "n_layer": 24,
  "n_positions": 2048,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "float32",
    "bnb_4bit_quant_type": "fp4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": false,
    "load_in_8bit": true
  },
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.0,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true

In [36]:
tokenizer = transformers.AutoTokenizer.from_pretrained('cerebras/Cerebras-GPT-1.3B')
#tokenizer.pad_token_id = 0

In [37]:
model = peft.PeftModel.from_pretrained(
    model,
    'new_trained',
    torch_dtype=torch.float16
)

In [53]:
generation_config = transformers.GenerationConfig(
    use_cache = True,
    do_sample = True,
    temperature = 0.1,
    top_p = 0.95,
    top_k = 80,
    repetition_penalty = 1.5,
    max_new_tokens = 120,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
)

In [54]:
def generate(input):
  inputs = tokenizer("Question: " + input + "\nAnswer:", return_tensors="pt")
  input_ids = inputs["input_ids"].to('cuda')

  with torch.no_grad():
    generation_output = model.generate(
        input_ids=input_ids,
        attention_mask=torch.ones_like(input_ids),
        generation_config=generation_config,
    )
    
  output_text = tokenizer.decode(generation_output[0].cuda(), skip_special_tokens=True).strip()
  return output_text

In [40]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install flask
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [55]:
!killall ngrok
!ngrok config add-authtoken 2QN9JlBY91WFv2EKuaFUCPCBsJC_7cK8YoEkQbjHj3pnnNpD5

ngrok: no process found
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
import os
import threading

from flask import Flask, render_template
from pyngrok import ngrok, conf
from flask import request

app = Flask(__name__)
port = 5000

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# ... Update inbound traffic via APIs to use the public-facing ngrok URL


# Define Flask routes
@app.route("/", methods=['GET', 'POST'])
def index():
    content_type = request.headers.get('Content-Type')
    if request.args.get('question'):
        answer = generate(request.args.get("question"))
        print(answer)
        return answer
    else:
        return 'Error'

if __name__ == '__main__':
    app.run()

INFO:werkzeug:127.0.0.1 - - [28/May/2023 12:32:28] "GET /?question=What+are+the+symptoms+of+having+flu? HTTP/1.1" 200 -


Question: What are the symptoms of having flu?
Answer: Symptoms can include fever, cough or runny nose and mouth. It's important to seek medical attention if you have any signs that indicate a serious infection such as bleeding in your throat, headache, muscle aches, fatigue, difficulty breathing, trouble swallowing or vomitting saliva from sneezing or coughing. If there is no evidence for an illness, it may be advisable to stay home and avoid sharing food with others until confirmed diagnosis has been made."


**DON'T RUN FROM HERE**

---



---



In [ ]:
!killall ngrok

In [ ]:
from pyngrok import ngrok

ngrok_process = ngrok.get_ngrok_process()
print(" Shutting down server.")
ngrok.kill()

In [ ]:
del model